### Importing Packages and API Key

In [42]:
# Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from scipy import stats

# Load environment variables from the .env file
load_dotenv('config.env')


True

In [43]:
# Import MlFinLab package
import mlfinlab
from mlfinlab.data_structures.standard_data_structures import (get_dollar_bars,
                                                               get_tick_bars, get_volume_bars)

from mlfinlab.multi_product.etf_trick import get_futures_roll_series
from mlfinlab.data_structures.imbalance_data_structures import get_ema_dollar_imbalance_bars


### Importing Data and Creating Dollar, Volume and Tick Bars

In [44]:
# import ES_Trades data
#path = 'Data/ES_Trades.csv'
path = 'https://raw.githubusercontent.com/jjakimoto/finance_ml/refs/heads/master/datasets/Google.csv'
data = pd.read_csv(path)

# viewing data
data.head()


,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
1,2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2,2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
3,2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
4,2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [45]:
# getting the min, max, mean and median of the volume column. The numbers should be in a format with commas separating the thousands and no decimal points.
min_volume = f'{data["Volume"].min():,}'
max_volume = f'{data["Volume"].max():,}'
mean_volume = f'{data["Volume"].mean():,.2f}'
median_volume = f'{data["Volume"].median():,}'

print('Min Volume:', min_volume)
print('Max Volume:', max_volume)
print('Mean Volume:', mean_volume)
print('Median Volume:', median_volume)

Min Volume: 521,141.0
Max Volume: 82,151,100.0
Mean Volume: 8,404,690.76
Median Volume: 5,620,700.0


In [46]:
# Format the Data

# renaming 'Date' to 'date'
data.rename(columns={'Date': 'date'}, inplace=True)

# renaming 'Adj. Close' to 'price'
data.rename(columns={'Adj. Close': 'price'}, inplace=True)

# renaming 'Volume' to 'volume'
data.rename(columns={'Volume': 'volume'}, inplace=True)

# selecting only the 'date', 'price' and 'volume' columns as the new data to be used
new_data = data[['date', 'price', 'volume']]
print(new_data.head())
print('\n')
print('Rows:', new_data.shape[0])

         date      price      volume
0  2004-08-19  50.322842  44659000.0
1  2004-08-20  54.322689  22834300.0
2  2004-08-23  54.869377  18256100.0
3  2004-08-24  52.597363  15247300.0
4  2004-08-25  53.164113   9188600.0


Rows: 3125


### Creating Dollar Bars and Dollar Imbalance Bars

In [47]:
# 'new_data' can be simply replaced with the file path where 'raw_tick_data' was saved if memory is an issue
print('Creating Dollar Bars')
dollar = get_dollar_bars(new_data, threshold=2000000000,
                         batch_size=1000, verbose=True)


Creating Dollar Bars
Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Returning bars 



In [48]:
# creating dollar imbalance bars
print('Creating Dollar Imbalance Bars')
dollar_imbalance = get_ema_dollar_imbalance_bars(new_data, num_prev_bars=3, expected_imbalance_window=10, exp_num_ticks_init=10, exp_num_ticks_constraints=[1, 10],
                                                 batch_size=1000, verbose=True, to_csv=False, analyse_thresholds=False, output_path=None)

Creating Dollar Imbalance Bars
Reading data in batches:
Batch number: 0
Batch number: 1
Batch number: 2
Batch number: 3
Returning bars 



### Confirming Sampling

In [49]:
# Confirm the dollar sampling
dollar['value'] = dollar['close'] * dollar['volume']
dollar.head()


,tick_num,open,high,low,close,volume,cum_buy_volume,cum_ticks,cum_dollar_value,value
date_time,,,,,,,,,,
2004-08-19,1,50.322842,50.322842,50.322842,50.322842,44659000.0,0.0,1,2.247368e+09,2.247368e+09
2004-08-23,3,54.322689,54.869377,54.322689,54.869377,41090400.0,41090400.0,2,2.242121e+09,2.254605e+09
2004-08-27,7,52.597363,54.122070,52.597363,53.239345,37742400.0,16283400.0,4,2.005164e+09,2.009381e+09
2004-09-03,12,51.162935,51.343492,50.159839,50.159839,39523700.0,20036400.0,5,2.006010e+09,1.982502e+09
2004-09-14,18,50.947269,55.917612,50.947269,55.917612,42266600.0,42266600.0,6,2.250130e+09,2.363447e+09


In [50]:
# Confirm the dollar imbalance sampling
dollar_imbalance['value'] = dollar_imbalance['close'] * dollar_imbalance['volume']
dollar_imbalance

,tick_num,open,high,low,close,volume,cum_buy_volume,cum_ticks,cum_dollar_value,value
date_time,,,,,,,,,,
2004-09-01,10,50.322842,54.869377,50.280210,50.280210,1.427445e+08,6.229160e+07,10,7.472499e+09,7.177223e+09
2004-09-02,11,50.912161,50.912161,50.912161,50.912161,1.511860e+07,1.511860e+07,1,7.697206e+08,7.697206e+08
2004-09-10,16,50.159839,52.828075,50.159839,52.828075,2.874600e+07,2.359360e+07,5,1.480121e+09,1.518596e+09
2004-09-14,18,53.916435,55.917612,53.916435,55.917612,1.867300e+07,1.867300e+07,2,1.028452e+09,1.044150e+09
2004-09-16,20,56.173402,57.161452,56.173402,57.161452,1.997930e+07,1.997930e+07,2,1.131461e+09,1.142046e+09
2004-09-20,22,58.926902,59.864797,58.926902,59.864797,2.010120e+07,2.010120e+07,2,1.194470e+09,1.203354e+09
2004-09-29,29,59.102444,65.742942,59.102444,65.742942,8.698040e+07,6.356220e+07,7,5.445390e+09,5.718347e+09
2004-10-14,40,65.000651,71.219849,65.000651,71.219849,1.477905e+08,9.910950e+07,11,1.014376e+10,1.052562e+10
2004-10-22,46,72.278116,86.481962,70.462511,86.481962,1.709232e+08,1.300908e+08,6,1.350710e+10,1.478177e+10


### Viewing the Dollar and Dollar Imbalance Bars

In [51]:
# linking the data to the naming convention below
continuous_dollar_relative_method = dollar
continuous_dollar_imbalance_relative_method = dollar_imbalance

# viewing the head of continuous_contract_relative_method
continuous_dollar_relative_method.head()


,tick_num,open,high,low,close,volume,cum_buy_volume,cum_ticks,cum_dollar_value,value
date_time,,,,,,,,,,
2004-08-19,1,50.322842,50.322842,50.322842,50.322842,44659000.0,0.0,1,2.247368e+09,2.247368e+09
2004-08-23,3,54.322689,54.869377,54.322689,54.869377,41090400.0,41090400.0,2,2.242121e+09,2.254605e+09
2004-08-27,7,52.597363,54.122070,52.597363,53.239345,37742400.0,16283400.0,4,2.005164e+09,2.009381e+09
2004-09-03,12,51.162935,51.343492,50.159839,50.159839,39523700.0,20036400.0,5,2.006010e+09,1.982502e+09
2004-09-14,18,50.947269,55.917612,50.947269,55.917612,42266600.0,42266600.0,6,2.250130e+09,2.363447e+09


In [52]:
# viewing the head of continuous_contract_relative_method for dollar imbalance
continuous_dollar_imbalance_relative_method.head()

,tick_num,open,high,low,close,volume,cum_buy_volume,cum_ticks,cum_dollar_value,value
date_time,,,,,,,,,,
2004-09-01,10,50.322842,54.869377,50.280210,50.280210,142744500.0,62291600.0,10,7.472499e+09,7.177223e+09
2004-09-02,11,50.912161,50.912161,50.912161,50.912161,15118600.0,15118600.0,1,7.697206e+08,7.697206e+08
2004-09-10,16,50.159839,52.828075,50.159839,52.828075,28746000.0,23593600.0,5,1.480121e+09,1.518596e+09
2004-09-14,18,53.916435,55.917612,53.916435,55.917612,18673000.0,18673000.0,2,1.028452e+09,1.044150e+09
2004-09-16,20,56.173402,57.161452,56.173402,57.161452,19979300.0,19979300.0,2,1.131461e+09,1.142046e+09


### Computing Serial correlation of the Dollar and Dollar Imbalance Bars

In [53]:
# computing the returns for the dollar bars and dollar imbalance bars using pct change
dollar_returns = continuous_dollar_relative_method['close'].pct_change().dropna()
dollar_imbalance_returns = continuous_dollar_imbalance_relative_method['close'].pct_change().dropna()

In [54]:
# computing the serial correlation for the dollar bars and dollar imbalance bars
dollar_serial_corr = dollar_returns.autocorr()
dollar_imbalance_serial_corr = dollar_imbalance_returns.autocorr()


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [55]:
# printing the serial correlation for the dollar bars and dollar imbalance bars
print('Serial Correlation for Dollar Bars:', dollar_serial_corr)
print('Serial Correlation for Dollar Imbalance Bars:', dollar_imbalance_serial_corr)

Serial Correlation for Dollar Bars: 0.023799572518800136
Serial Correlation for Dollar Imbalance Bars: -0.2563899257520614
